In [7]:
firstime = False

In [8]:
folder_path = '/content/drive/My Drive/University/SEM7/CS4642_Data_Mining_and_Information_Retrieval/project_DM_Assignment/'

# Import Statements

In [9]:
if (firstime):
  from google.colab import drive
  drive.mount('/content/drive')
  !pip install catboost
from pathlib import Path
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score,roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

import xgboost as xgb
from catboost import CatBoostClassifier

pd.set_option("display.max_columns", 100)

# Load Data

In [10]:
features1_df = pd.read_csv(
    folder_path + "training_set_features.csv", 
    index_col="respondent_id"
)
labels_df = pd.read_csv(
    folder_path + "training_set_labels.csv", 
    index_col="respondent_id"
)
test_features_df = pd.read_csv(folder_path + "test_set_features.csv", 
                               index_col="respondent_id")

In [11]:
frames = [features1_df, test_features_df]
features_df = pd.concat(frames)

# Feature Engineering

## Fill null values

In [12]:
features_df['age_group'] = features_df['age_group'].fillna(features_df['age_group'].mode().iloc[0])
features_df['education'] = features_df['education'].fillna(features_df['education'].mode().iloc[0])
features_df['sex'] = features_df['sex'].fillna(features_df['sex'].mode().iloc[0])
features_df['race'] = features_df['race'].fillna(features_df['race'].mode().iloc[0])
features_df['income_poverty'] = features_df['income_poverty'].fillna(features_df['income_poverty'].mode().iloc[0])
features_df['marital_status'] = features_df['marital_status'].fillna(features_df['marital_status'].mode().iloc[0])
features_df['rent_or_own'] = features_df['rent_or_own'].fillna(features_df['rent_or_own'].mode().iloc[0])
features_df['employment_status'] = features_df['employment_status'].fillna(features_df['employment_status'].mode().iloc[0])
features_df['hhs_geo_region'] = features_df['hhs_geo_region'].fillna(features_df['hhs_geo_region'].mode().iloc[0])
features_df['census_msa'] = features_df['census_msa'].fillna(features_df['census_msa'].mode().iloc[0])
features_df['employment_industry'] = features_df['employment_industry'].fillna(features_df['employment_industry'].mode().iloc[0])
features_df['employment_occupation'] = features_df['employment_occupation'].fillna(-1)

In [13]:
features_df['h1n1_concern'] = features_df['h1n1_concern'].fillna(features_df['h1n1_concern'].mode().iloc[0])
features_df['h1n1_knowledge'] = features_df['h1n1_knowledge'].fillna(features_df['h1n1_knowledge'].mode().iloc[0])
features_df['opinion_h1n1_vacc_effective'] = features_df['opinion_h1n1_vacc_effective'].fillna(features_df['opinion_h1n1_vacc_effective'].mode().iloc[0])
features_df['opinion_h1n1_risk'] = features_df['opinion_h1n1_risk'].fillna(features_df['opinion_h1n1_risk'].mode().iloc[0])
features_df['opinion_h1n1_sick_from_vacc'] = features_df['opinion_h1n1_sick_from_vacc'].fillna(features_df['opinion_h1n1_sick_from_vacc'].mode().iloc[0])
features_df['opinion_seas_vacc_effective'] = features_df['opinion_seas_vacc_effective'].fillna(features_df['opinion_seas_vacc_effective'].mode().iloc[0])
features_df['opinion_seas_risk'] = features_df['opinion_seas_risk'].fillna(features_df['opinion_seas_risk'].mode().iloc[0])
features_df['opinion_seas_sick_from_vacc'] = features_df['opinion_seas_sick_from_vacc'].fillna(features_df['opinion_seas_sick_from_vacc'].mode().iloc[0])

## Factorize values

In [14]:
features_df['age_group'] = pd.factorize(features_df.age_group)[0]
features_df['age_group'] = features_df['age_group'].astype("int")
features_df['education'] = pd.factorize(features_df.education)[0]
features_df['education'] = features_df['education'].astype("int")
features_df['sex'] = pd.factorize(features_df.sex)[0]
features_df['sex'] = features_df['sex'].astype("int")
features_df['race'] = pd.factorize(features_df.race)[0]
features_df['race'] = features_df['race'].astype("int")
features_df['income_poverty'] = pd.factorize(features_df.income_poverty)[0]
features_df['income_poverty'] = features_df['income_poverty'].astype("int")
features_df['marital_status'] = pd.factorize(features_df.marital_status)[0]
features_df['marital_status'] = features_df['marital_status'].astype("int")
features_df['rent_or_own'] = pd.factorize(features_df.rent_or_own)[0]
features_df['rent_or_own'] = features_df['rent_or_own'].astype("int")
features_df['employment_status'] = pd.factorize(features_df.employment_status)[0]
features_df['employment_status'] = features_df['employment_status'].astype("int")
features_df['hhs_geo_region'] = pd.factorize(features_df.hhs_geo_region)[0]
features_df['hhs_geo_region'] = features_df['hhs_geo_region'].astype("int")
features_df['census_msa'] = pd.factorize(features_df.census_msa)[0]
features_df['census_msa'] = features_df['census_msa'].astype("int")
features_df['employment_industry'] = pd.factorize(features_df.employment_industry)[0]
features_df['employment_industry'] = features_df['employment_industry'].astype("int")
features_df['employment_occupation'] = pd.factorize(features_df.employment_occupation)[0]
features_df['employment_occupation'] = features_df['employment_occupation'].astype("int")

In [15]:
features_df = features_df.fillna(-1)
features_df = features_df.astype('int')
features_df.shape

(53415, 35)

## Split train/Test

In [16]:
features1_df, test_features_df = features_df[0:26707], features_df[26707:]

## Merge train features with labels

In [17]:
features_df2 = pd.merge(features1_df,labels_df,on ='respondent_id',sort =True)

## Drop null valuess

In [18]:
features_df2 = features_df2.dropna()

## Seperate h1n1/seasonal features

In [19]:
features_h1n1 = features_df2.drop(columns=['h1n1_vaccine','seasonal_vaccine'])
features_seasonal = features_df2.drop(columns=['h1n1_vaccine','seasonal_vaccine'])

In [20]:
labels_h1n1 = features_df2["h1n1_vaccine"]
labels_seasonal = features_df2["seasonal_vaccine"]

In [21]:
test_features_df_h1n1 = test_features_df
test_features_df_seasonal = test_features_df

# Hyperparameter Tunning

In [22]:

def do_hyperparameter_tunning(features_seasonal,labels_seasonal):

  train_data = features_seasonal
  train_labels = labels_seasonal

  model = CatBoostClassifier(loss_function='Logloss')

  grid = {'learning_rate': [0.03, 0.1],
          'depth': [4, 6, 10],
          'l2_leaf_reg': [1, 3, 5, 7, 9]}

  grid_search_result = model.grid_search(grid, 
                                        X=train_data, 
                                        y=train_labels, 
                                        plot=True)
  return grid_search_result

log_loss
h1n1 : 'params': {'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.1}}
seasonal :'params': {'depth': 6, 'l2_leaf_reg': 7, 'learning_rate': 0.03}}

# Model for H1N1

In [23]:
categorical_var_h1n1 = np.where(features_h1n1.dtypes != np.float)[0]
print('\nCategorical Variables indices : ',categorical_var_h1n1)


Categorical Variables indices :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]


In [24]:
model_h1n1 = CatBoostClassifier(iterations=5000,cat_features=categorical_var_h1n1, nan_mode='Max', depth=8, l2_leaf_reg=3, learning_rate=0.1)

In [25]:
model_h1n1.fit(features_h1n1,labels_h1n1,plot=False)

Streaming output truncated to the last 5000 lines.
1:	learn: 0.5603228	total: 427ms	remaining: 17m 47s
2:	learn: 0.5182275	total: 607ms	remaining: 16m 50s
3:	learn: 0.4831331	total: 754ms	remaining: 15m 41s
4:	learn: 0.4569961	total: 912ms	remaining: 15m 10s
5:	learn: 0.4348032	total: 1.15s	remaining: 16m
6:	learn: 0.4181698	total: 1.37s	remaining: 16m 20s
7:	learn: 0.4050689	total: 1.59s	remaining: 16m 32s
8:	learn: 0.3944414	total: 1.84s	remaining: 17m 1s
9:	learn: 0.3860893	total: 2.08s	remaining: 17m 17s
10:	learn: 0.3788471	total: 2.29s	remaining: 17m 20s
11:	learn: 0.3728715	total: 2.52s	remaining: 17m 28s
12:	learn: 0.3684804	total: 2.75s	remaining: 17m 36s
13:	learn: 0.3653910	total: 3s	remaining: 17m 47s
14:	learn: 0.3621899	total: 3.26s	remaining: 18m 2s
15:	learn: 0.3590905	total: 3.52s	remaining: 18m 15s
16:	learn: 0.3566974	total: 3.74s	remaining: 18m 17s
17:	learn: 0.3544389	total: 3.99s	remaining: 18m 24s
18:	learn: 0.3527399	total: 4.25s	remaining: 18m 34s
19:	learn: 0.

# Model for seasonal fever 

In [26]:
categorical_var_seasonal = np.where(features_seasonal.dtypes != np.float)[0]
print('\nCategorical Variables indices : ',categorical_var_seasonal)


Categorical Variables indices :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]


In [27]:
model_seasonal = CatBoostClassifier(iterations=5000,nan_mode='Max',cat_features=categorical_var_seasonal, depth=8, l2_leaf_reg=7, learning_rate=0.03)

In [ ]:
model_seasonal.fit(features_seasonal,labels_seasonal,plot=False)

0:	learn: 0.6796439	total: 199ms	remaining: 16m 32s
1:	learn: 0.6670418	total: 413ms	remaining: 17m 11s
2:	learn: 0.6556874	total: 607ms	remaining: 16m 50s
3:	learn: 0.6446457	total: 790ms	remaining: 16m 27s
4:	learn: 0.6349575	total: 989ms	remaining: 16m 27s
5:	learn: 0.6252628	total: 1.2s	remaining: 16m 37s
6:	learn: 0.6169314	total: 1.38s	remaining: 16m 27s
7:	learn: 0.6088882	total: 1.56s	remaining: 16m 14s
8:	learn: 0.6014312	total: 1.78s	remaining: 16m 29s
9:	learn: 0.5940504	total: 2.03s	remaining: 16m 53s
10:	learn: 0.5870139	total: 2.25s	remaining: 17m 1s
11:	learn: 0.5805483	total: 2.44s	remaining: 16m 54s
12:	learn: 0.5746436	total: 2.64s	remaining: 16m 54s
13:	learn: 0.5687866	total: 2.85s	remaining: 16m 54s
14:	learn: 0.5636828	total: 3.05s	remaining: 16m 53s
15:	learn: 0.5586575	total: 3.27s	remaining: 16m 57s
16:	learn: 0.5538802	total: 3.45s	remaining: 16m 50s
17:	learn: 0.5492580	total: 3.65s	remaining: 16m 49s
18:	learn: 0.5456257	total: 3.85s	remaining: 16m 48s
19:	l

# Get Predictions

In [ ]:
predict_h1n1 = model_h1n1.predict_proba(test_features_df_h1n1)
predict_seasonal = model_seasonal.predict_proba(test_features_df_seasonal)

In [ ]:
submission_df = pd.read_csv(folder_path + "submission_format.csv", 
                            index_col="respondent_id")

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = predict_h1n1[:, 1]
submission_df["seasonal_vaccine"] = predict_seasonal[:, 1]

submission_df.head()

In [ ]:
submission_df.to_csv(folder_path + 'Submissions/catboost_5000epochs.csv', index=True)